# UrbanSound8k exploración de datos

---



## Intalar paquetes

Se instala:
- Procesar archivos de audio: `librosa`, `mutagen`
- Graficar: `Plotly`, `matplotlib`

In [ ]:
!pip install pandas
!pip install librosa
!pip install plotly
!pip install matplotlib
!pip install mutagen
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
import time
import librosa
import zipfile
import mutagen
import mutagen.wave
import numpy as np
import pandas as pd
import librosa.display
import IPython.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image

## Procesamiento de los datos

In [ ]:
# Unzip dataset
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O urban8k.tgz
!tar -xzf urban8k.tgz
!rm urban8k.tgz

--2024-10-30 00:31:26--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1203745/files/UrbanSound8K.tar.gz [following]
--2024-10-30 00:31:27--  https://zenodo.org/records/1203745/files/UrbanSound8K.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘urban8k.tgz’

urban8k.tgz         100%[===================>]   5.61G  30.0MB/s    in 3m 19s  

2024-10-30 00:34:46 (28.9 MB/s) - ‘urban8k.tgz’ saved [6023741708/6023741708]



In [ ]:
!cat UrbanSound8K/UrbanSound8K_README.txt

UrbanSound8K

Created By
----------

Justin Salamon*^, Christopher Jacoby* and Juan Pablo Bello*
* Music and Audio Research Lab (MARL), New York University, USA
^ Center for Urban Science and Progress (CUSP), New York University, USA
http://serv.cusp.nyu.edu/projects/urbansounddataset
http://marl.smusic.nyu.edu/
http://cusp.nyu.edu/

Version 1.0


Description
-----------

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, 
children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, and street_music. The classes are 
drawn from the urban sound taxonomy described in the following article, which also includes a detailed description of 
the dataset and how it was compiled:

J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 
22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.

All excerpts are taken from field recordings uploaded to www.freesou

## Análisis de datos
Queremos ver cómo se grabaron los archivos de audio y cómo se distribuyen las diferentes clases.




In [ ]:
def create_dataset_df(csv_file):
    dataset_df = pd.read_csv(csv_file)
    # Genera las rutas completas de archivo directamente con una expresión
    dataset_df['filepath'] = dataset_df.apply(
        lambda row: os.path.join('UrbanSound8K/audio', f"fold{row['fold']}", row['slice_file_name']), axis=1
    )
    return dataset_df

In [ ]:
dataset_df = create_dataset_df('UrbanSound8K/metadata/UrbanSound8K.csv')
dataset_df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,filepath
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,UrbanSound8K/audio/fold5/100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-137.wav


In [ ]:
dataset_df.groupby('class').slice_file_name.count()

,slice_file_name
class,
air_conditioner,1000
car_horn,429
children_playing,1000
dog_bark,1000
drilling,1000
engine_idling,1000
gun_shot,374
jackhammer,1000
siren,929


Podemos observar que todas las clases, excepto `car_horn`, `gun_shot` y `siren`, tienen 1000 muestras. Podríamos explorar opciones para incluir **pesos de clase** en la función de pérdida o **sobremuestreo** si vemos que las clases subrepresentadas no se clasifican tan bien como el resto. Tambien existe la opción de no trabajar con clases como `car_horn` y `gun_shot` que son las mas desiquilibradas y simplemente trabajar bajo el numero de muestras de `siren`.

In [ ]:
# Filtrar las clases para mantener solo aquellas con un número suficiente de muestras
dataset_df_filter = dataset_df[dataset_df['class'].isin([
    'air_conditioner', 'children_playing','dog_bark', 'drilling', 'engine_idling', 'jackhammer', 'siren', 'street_music'])]

In [ ]:
# Crear un DataFrame vacío para almacenar el dataset balanceado
dataset_balance = pd.DataFrame()

# Iterar sobre las clases seleccionadas
for class_name in dataset_df_filter['class'].unique():  # Usar unique() para obtener los nombres de clase
    # Filtrar las muestras de la clase actual
    class_samples = dataset_df_filter[dataset_df_filter['class'] == class_name]

    # Seleccionar aleatoriamente 929 muestras
    if len(class_samples) > 929:
        class_samples = class_samples.sample(n=929, random_state=42)

    # Agregar las muestras seleccionadas al nuevo DataFrame
    dataset_balance = pd.concat([dataset_balance, class_samples], ignore_index=True)

In [ ]:
# Mostrar el conteo de muestras por clase
dataset_balance.groupby('class').slice_file_name.count()

,slice_file_name
class,
air_conditioner,929
children_playing,929
drilling,929
engine_idling,929
jackhammer,929
siren,929
street_music,929


Next we compute the audio files statistics...

In [ ]:
dataset_balance.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,filepath
0,187110-2-0-27.wav,187110,13.500000,17.500000,1,10,2,children_playing,UrbanSound8K/audio/fold10/187110-2-0-27.wav
1,31150-2-0-1.wav,31150,0.500000,4.500000,2,5,2,children_playing,UrbanSound8K/audio/fold5/31150-2-0-1.wav
2,31150-2-0-2.wav,31150,1.000000,5.000000,2,5,2,children_playing,UrbanSound8K/audio/fold5/31150-2-0-2.wav
3,197554-2-0-74.wav,197554,47.850495,51.850495,1,10,2,children_playing,UrbanSound8K/audio/fold10/197554-2-0-74.wav
4,175296-2-0-3.wav,175296,1.500000,5.500000,1,7,2,children_playing,UrbanSound8K/audio/fold7/175296-2-0-3.wav


In [ ]:
dataset_new = dataset_balance.drop(columns=['fold', 'slice_file_name', 'fsID', 'start', 'end'])

In [ ]:
# Función para obtener la duración de un archivo de audio
def get_duration(file_path):
    try:
        duration = librosa.get_duration(filename=file_path)
        return duration
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Añadir una columna de duración al DataFrame
dataset_new['duration'] = dataset_new['filepath'].apply(get_duration)

# Filtrar el DataFrame para quedarte solo con audios entre 3 y 4 segundos
dataset_final = dataset_new[(dataset_new['duration'] >= 3) & (dataset_new['duration'] <= 4)]

# Visualizar los primeros resultados
dataset_final.head()

<ipython-input-13-8c2ed86279a8>:4: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


,salience,classID,class,filepath,duration
0,1,2,children_playing,UrbanSound8K/audio/fold10/187110-2-0-27.wav,4.0
1,2,2,children_playing,UrbanSound8K/audio/fold5/31150-2-0-1.wav,4.0
2,2,2,children_playing,UrbanSound8K/audio/fold5/31150-2-0-2.wav,4.0
3,1,2,children_playing,UrbanSound8K/audio/fold10/197554-2-0-74.wav,4.0
4,1,2,children_playing,UrbanSound8K/audio/fold7/175296-2-0-3.wav,4.0


In [ ]:
# Mostrar el conteo de muestras por clase
dataset_final.groupby('class').filepath.count()

,filepath
class,
air_conditioner,927
children_playing,902
drilling,766
engine_idling,904
jackhammer,778
siren,901
street_music,929


In [ ]:
# Crear un nuevo DataFrame vacío para almacenar el dataset balanceado
dataset_balanced_final = pd.DataFrame(columns=dataset_final.columns)

# Iterar sobre cada clase en el DataFrame
for class_name in dataset_final['class'].unique():
    # Filtrar las muestras de la clase actual
    class_samples = dataset_final[dataset_final['class'] == class_name]

    # Seleccionar aleatoriamente 766 muestras si hay más, o todas si hay menos o igual a 766
    if len(class_samples) > 766:
        class_samples = class_samples.sample(n=766, random_state=42)
    elif len(class_samples) < 766:
        print(f"Advertencia: La clase {class_name} tiene menos de 766 muestras y no se puede completar.")

    # Agregar las muestras seleccionadas al nuevo DataFrame
    dataset_balanced_final = pd.concat([dataset_balanced_final, class_samples], ignore_index=True)

<ipython-input-15-7f79c4abed00>:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_balanced_final = pd.concat([dataset_balanced_final, class_samples], ignore_index=True)


In [ ]:
# Mostrar el conteo de muestras por clase
dataset_balanced_final.groupby('class').filepath.count()

,filepath
class,
air_conditioner,766
children_playing,766
drilling,766
engine_idling,766
jackhammer,766
siren,766
street_music,766


In [ ]:
import soundfile as sf
# Definir la duración deseada en segundos
target_duration = 4  # segundos

# Iterar sobre cada muestra en el DataFrame
for index, row in dataset_balanced_final.iterrows():
    file_path = row['filepath']

    # Cargar el audio
    audio, sr = librosa.load(file_path, sr=None)  # Mantener la frecuencia de muestreo original

    # Calcular la duración actual
    current_duration = librosa.get_duration(y=audio, sr=sr)

    # Verificar si el audio es menor a la duración objetivo
    if current_duration < target_duration:
        # Calcular la cantidad de muestras para alcanzar los 4 segundos
        target_samples = int(target_duration * sr)
        padding_needed = target_samples - len(audio)

        # Aplicar padding (relleno) con ceros (silencio)
        padded_audio = np.pad(audio, (0, padding_needed), 'constant')

        # Guardar el audio con padding
        sf.write(file_path, padded_audio, sr)  # Sobrescribe el archivo original

In [ ]:
# Crear una copia del DataFrame para agregar la columna de duración actualizada
dataset_balanced_finalV2 = dataset_balanced_final.copy()

# Función para calcular la duración del archivo de audio
def calculate_duration(file_path):
    try:
        duration = librosa.get_duration(filename=file_path)
        return duration
    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return None

# Sobrescribir la columna 'duration' en el DataFrame con las duraciones actualizadas
dataset_balanced_finalV2['duration'] = dataset_balanced_finalV2['filepath'].apply(calculate_duration)

# Mostrar los primeros resultados para verificar que se haya actualizado correctamente
dataset_balanced_finalV2.head()


<ipython-input-30-e3ca6f75378b>:7: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


,salience,classID,class,filepath,duration
0,2,2,children_playing,UrbanSound8K/audio/fold1/135776-2-0-32.wav,4.0
1,2,2,children_playing,UrbanSound8K/audio/fold3/88569-2-0-14.wav,4.0
2,1,2,children_playing,UrbanSound8K/audio/fold10/101382-2-0-29.wav,4.0
3,1,2,children_playing,UrbanSound8K/audio/fold8/204526-2-0-166.wav,4.0
4,1,2,children_playing,UrbanSound8K/audio/fold6/116423-2-0-4.wav,4.0


#Guardo el nuevo directorio con las clases balanceadas

En el directorio se van a organizar los diferentes audios en las clases que le corresponde, esto para facilitar la manipulación de los datos y mantener un orden para obtener los espectrogramas

In [ ]:
import os
import shutil
# Definir el directorio de salida
output_dir = 'UrbanSound8k_Processing'
audio_output_dir = os.path.join(output_dir, 'Audio')
metadata_output_dir = os.path.join(output_dir, 'Metadata')

# Crear las carpetas principales
os.makedirs(audio_output_dir, exist_ok=True)
os.makedirs(metadata_output_dir, exist_ok=True)

# Guardar el DataFrame "dataset_new" como CSV en la carpeta de Metadata
dataset_balanced_finalV2.to_csv(os.path.join(metadata_output_dir, 'UrbanSound8k.csv'), index=False)

# Iterar sobre cada fila del DataFrame "dataset_new"
for _, row in dataset_balanced_finalV2.iterrows():
    class_name = row['class']
    src_filepath = row['filepath']
    class_dir = os.path.join(audio_output_dir, class_name)

    # Crear la carpeta para la clase si no existe
    os.makedirs(class_dir, exist_ok=True)

    # Copiar el archivo al nuevo directorio con la estructura organizada
    shutil.copy(src_filepath, class_dir)

print("Organización de archivos completada.")

Organización de archivos completada.


In [ ]:
# Comprimir la carpeta UrbanSound8k_Processing en un archivo ZIP
shutil.make_archive('UrbanSound8k_Processing', 'zip', output_dir)

'/content/UrbanSound8k_Processing.zip'